In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/My Drive/Colab Notebooks/Project3_Face_Recognition_with_SVM
!ls

/content/drive/My Drive/Colab Notebooks/Project3_Face_Recognition_with_SVM
a1_TrainingPhotos  b2_FR_Predictor.ipynb     d_FR_Client_Deck.pdf
a2_NewPhotos	   c1_FR_PCA.pkl
b1_FR_Model.ipynb  c2_FR_svm_classifier.pkl


In [7]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

In [10]:
folder='a1_TrainingPhotos/' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image

    img=img.resize([32,64])  # resize image  to a fixed size

    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, multichannel=True) # obtain HOG features

    X.append(fd)

    # obtain training labels from file name

    if re.match("Amir*", filename):
      Yt=0
    elif re.match("Jaya*", filename):
      Yt=1
    elif re.match("Hir*", filename):
      Yt=2
    elif re.match("Anupam*", filename):
      Yt=3
    else:
      Yt=4

    Y.append(Yt)

    i=i+1


In [11]:
fd.shape

(3780,)

In [12]:
target_names=['AamirKhan', 'JayaBadhuri', 'HritikRoshan', 'AnupamKher']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [14]:
n_components = 25
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [15]:
# project training and test data to lower dimension
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [16]:
# Save PCA coefficients for use at prediction

import pickle as pk
pk.dump(pca, open("c1_FR_PCA.pkl","wb"))

In [17]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.1, probability=True)


In [18]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_FR_svm_classifier.pkl']

In [19]:
y_pred = clf.predict(X_test_pca)
y_pred

array([0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 2, 1, 2, 2, 3, 2, 0, 0, 1, 0, 0, 1, 0, 2, 1, 1, 0, 1, 2,
       0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 3, 0, 0, 1,
       0, 1, 3, 0, 0, 1, 2, 3, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 3, 1, 0, 1, 1, 0, 2, 0, 0, 0, 3, 2, 0, 0])

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.98      0.85        42
           1       0.94      0.85      0.89        40
           2       0.91      0.56      0.69        18
           3       1.00      0.86      0.92         7

    accuracy                           0.85       107
   macro avg       0.90      0.81      0.84       107
weighted avg       0.87      0.85      0.85       107

